---
title: "Naive Bayes"
format:
  html:
    toc: true
---

## Introduction

Naive Bayes(NB) classification is a probabilistic machine learning technique based on Bayes' theorem. It's widely used for various classification tasks, such as spam detection, sentiment analysis, and more. Its primary objective is to predict the class or category of a given data point based on the observed features. For the purpose of this study, I will perform NB on a recorded dataset(`Cali20_climate_fire`) and a text dataset(`articles_with_sentiment`).

### How Naive Bayes Works:

1. **Probabilistic Nature**: Naive Bayes is probabilistic in nature because it estimates the probability of a data point belonging to a particular class or category. It does this by calculating the conditional probabilities of the features given the class.

2. **Bayes' Theorem Foundation**: It's built upon Bayes' theorem, which describes how to update the probability for a hypothesis based on new evidence. In classification, it's used to calculate the probability of a class given the observed features. The "Naive" part of Naive Bayes is the assumption that the features are conditionally independent, which simplifies the calculations.

### Objectives of Naive Bayes Classification:

In both datasets, the objective is to classify or predict something based on the given features. In the first dataset, I aim to predict the occurrence of a fire based on weather variable precipitation. In the second dataset, I want to classify the sentiment of text based on the content of the article.

### Different Variants of Naive Bayes:

1. **Gaussian Naive Bayes**: Use this when features are continuous and have a Gaussian (normal) distribution. 
2. **Multinomial Naive Bayes**: Suitable for discrete data, often used for text classification with features like word counts. I am using this model predicting sentiments basing on the contents of the articles for the text dataset.
3. **Bernoulli Naive Bayes**: Appropriate when dealing with binary or Boolean features. It's often used for binary text classification where the presence or absence of words matters. 